# Prepare data for Pipeline

## Setup and settings

In [1]:
# Importing packages
import os
import yaml
import logging
import pandas as pd
from Bio.Seq import Seq
from qiime2 import Artifact, Visualization, Metadata
from qiime2.plugins import demux
from qiime2.plugins.cutadapt.methods import trim_paired, trim_single
from qiime2.plugins.demux.methods import filter_samples

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
params_path = os.path.join('..', 'params', 'parameter-set-01.yaml')
experiment_name = 'exp-01'
base_dir = os.path.join('/', 'home', 'username', 'pipeline-dir')
manifest_file = os.path.join(base_dir, 'data', 'raw', 'manifest', 'manifest.csv')
replace_files = False
trim = None
metadata_file = None
threads = 1
demux_file = None

In [3]:
# Parameters
base_dir = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri"
class_col = "group-id"
classifier_file = "/mnt/nupeb/rede-micro/datasets/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
experiment_name = "ana-flavia-NRxSTD-NR-trim"
manifest_file = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest/manifest-ana-flavia-NRxSTD-NR.csv"
metadata_file = "/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-NRxSTD-NR.tsv"
overlap = 12
phred = 20
replace_files = False
threads = 6
top_n = 20
trim = {
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "overlap": 8,
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}
trunc_f = 0
trunc_r = 0


In [4]:
out_dir = os.path.join(base_dir, 'experiments', experiment_name, 'qiime-artifacts')
img_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name, 'imgs'))
data_dir =  os.path.abspath(os.path.join(base_dir, 'data'))
raw_data_dir =  os.path.abspath(os.path.join(data_dir, 'raw'))
interim_data_dir =  os.path.abspath(os.path.join(data_dir, 'interim'))

### Defining names and paths

In [5]:
# Create path if it not exists
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
    print(f'New qiime-artifacts folder path created: {out_dir}')

# Create path if it not exists
if not os.path.isdir(img_folder):
    os.makedirs(img_folder)
    print(f'New img folder path created: {img_folder}')

New qiime-artifacts folder path created: /mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-NRxSTD-NR-trim/qiime-artifacts
New img folder path created: /mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-NRxSTD-NR-trim/imgs


In [6]:
manifest_df = pd.read_csv(manifest_file)
n_directions = len(manifest_df['direction'].unique())
if n_directions == 1:
    d_type = 'SampleData[SequencesWithQuality]'
    v_type = 'SingleEndFastqManifestPhred33'
    direction = 'single'
elif n_directions == 2:
    d_type = 'SampleData[PairedEndSequencesWithQuality]'
    v_type = 'PairedEndFastqManifestPhred33'
    direction = 'paired'
else:
    print(f'ERROR: invalid number of directions {n_directions}')

In [7]:
sufix = str(direction)
if trim and len(trim.keys()) == 3:
    sufix += f'-{trim}'
    
# Define the output artifact full path
if demux_file is None:
    demux_file = os.path.join(out_dir, f'demux-{direction}.qza')
demux_view = os.path.join(out_dir, f'demux-{direction}.qzv')
demux_file_trim = os.path.join(out_dir, f'demux-{direction}-trim.qza')
demux_view_trim = os.path.join(out_dir, f'demux-{direction}-trim.qzv')

## Step execution

This Step import all `fastq` files in a **QIIME2 Artifact** object and save it to a new `qza` file.

In [8]:
# If the metadata file is not defined, use the default metadata file
metadata_qa = Metadata.load(metadata_file)

In [9]:
# If the demux file does not exist or if the old file will be replaced
if not os.path.isfile(demux_file) or replace_files:
    
    interim_demux_path = os.path.join(interim_data_dir, f'demux-{direction}.qza')
    
    # Create new file path for interim dir
    if not os.path.isdir(interim_data_dir):
        !mkdir -p {interim_data_dir}
        
    # Load Artifact from interim folder
    if os.path.isfile(interim_demux_path):
        artifact = Artifact.load(interim_demux_path)
        
    # Create new Artifact using Manifest
    else:
        artifact = Artifact.import_data(d_type, manifest_file, view_type=v_type)
        if not os.path.isfile(interim_demux_path):
            artifact.save(interim_demux_path)
            
    # Filter and Save Artifact
    artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
    artifact.save(demux_file)
    
    # Visualization
    if os.path.isfile(demux_view):
        demux_view_obj = Visualization.load(demux_view)
    else:
        demux_view_obj = demux.visualizers.summarize(artifact).visualization
        Visualization.save(demux_view_obj, filepath=demux_view)
        
elif os.path.isfile(demux_file):
    
    # Load Artifact
    artifact = Artifact.load(demux_file)
    artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
    
    # Visualization
    if os.path.isfile(demux_view):
        demux_view_obj = Visualization.load(demux_view)
    else:
        demux_view_obj = demux.visualizers.summarize(artifact).visualization
        Visualization.save(demux_view_obj, filepath=demux_view)

/home/lauro/anaconda3/envs/qiime2-2023.5/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)


/home/lauro/anaconda3/envs/qiime2-2023.5/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)


## Step report


In [10]:
print(demux_view_obj)

# Render Visualization
demux_view_obj

<visualization: Visualization uuid: 2e477aba-0d92-46b9-b138-ea501820aac3>


<visualization: Visualization uuid: 2e477aba-0d92-46b9-b138-ea501820aac3>

[cutadapt](https://docs.qiime2.org/2022.2/plugins/available/cutadapt/trim-paired/)

We trim the forward primer and the reverse complement of the reverse primer from the forward reads. We trim the reverse primer and reverse complement of the forward primer from the reverse reads.

In [11]:
demux_file_trim = os.path.join(out_dir, f'demux-{direction}-trim.qza')
demux_view_trim = os.path.join(out_dir, f'demux-{direction}-trim.qzv')

if trim and (not os.path.isfile(demux_file_trim) or replace_files):
    forward_primer = [trim['forward_primer']] # ['CCTACGGGRSGCAGCAG']
    reverse_primer = [trim['reverse_primer']] # ['GGACTACHVGGGTWTCTAAT']
    forward_reverse_complement = [str(Seq(forward_primer[0]).reverse_complement())]
    reverse_reverse_complement = [str(Seq(reverse_primer[0]).reverse_complement())]

    res = None
    
    if n_directions == 1:
        res = trim_single(
            demultiplexed_sequences=artifact,
            front=forward_primer,
            adapter=reverse_reverse_complement,
            cores=threads,
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    elif n_directions == 2:
        res = trim_paired(
            demultiplexed_sequences=artifact,
            front_f=forward_primer,
            front_r=reverse_primer,
            # adapter_f=reverse_reverse_complement,
            # adapter_r=forward_reverse_complement,
            cores=threads,
            overlap=trim['overlap'],
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    else:
        print(f'ERROR: invalid number of directions {n_directions}')

    if res is not None:
        res.save(demux_file_trim)
        Visualization.save(demux.visualizers.summarize(res).visualization, filepath=demux_view_trim)

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -q 0,0 --quality-base 33 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-mcqlaf4n/S210421121677_8_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-mcqlaf4n/S210421121677_9_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2/lauro/data/1dff5a72-bbd8-47cf-8f76-5a4793729825/data/S210421121677_8_L001_R1_001.fastq.gz /tmp/qiime2/lauro/data/1dff5a72-bbd8-47cf-8f76-5a4793729825/data/S210421121677_9_L001_R2_001.fastq.gz

This is cutadapt 4.4 with Python 3.8.16
Command line parameters: --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -q 0,0 --quality-base

Finished in 1.303 s (3.566 µs/read; 16.82 M reads/minute).

=== Summary ===

Total read pairs processed:            365,427
  Read 1 with adapter:                 355,998 (97.4%)
  Read 2 with adapter:                 339,330 (92.9%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           34,713 (9.5%)
Pairs written (passing filters):       330,714 (90.5%)

Total basepairs processed:   186,367,770 bp
  Read 1:   111,455,235 bp
  Read 2:    74,912,535 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):    154,181,570 bp (82.7%)
  Read 1:    94,107,785 bp
  Read 2:    60,073,785 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 355998 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	1	5.6	0	1
9	2	1.4	0	2
10	

Finished in 0.397 s (4.145 µs/read; 14.48 M reads/minute).

=== Summary ===

Total read pairs processed:             95,852
  Read 1 with adapter:                  93,250 (97.3%)
  Read 2 with adapter:                  88,539 (92.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            9,674 (10.1%)
Pairs written (passing filters):        86,178 (89.9%)

Total basepairs processed:    48,884,520 bp
  Read 1:    29,234,860 bp
  Read 2:    19,649,660 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):     40,178,039 bp (82.2%)
  Read 1:    24,522,719 bp
  Read 2:    15,655,320 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 93250 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
9	5	0.4	0	5
10	1	0.1	0	1
11

Finished in 1.198 s (3.780 µs/read; 15.87 M reads/minute).

=== Summary ===

Total read pairs processed:            316,849
  Read 1 with adapter:                 308,351 (97.3%)
  Read 2 with adapter:                 292,785 (92.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           31,765 (10.0%)
Pairs written (passing filters):       285,084 (90.0%)

Total basepairs processed:   161,592,990 bp
  Read 1:    96,638,945 bp
  Read 2:    64,954,045 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):    132,905,965 bp (82.2%)
  Read 1:    81,120,487 bp
  Read 2:    51,785,478 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 308351 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	4	4.8	0	4
9	1	1.2	0	1
10

Finished in 1.124 s (4.521 µs/read; 13.27 M reads/minute).

=== Summary ===

Total read pairs processed:            248,647
  Read 1 with adapter:                 241,815 (97.3%)
  Read 2 with adapter:                 229,975 (92.5%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           24,848 (10.0%)
Pairs written (passing filters):       223,799 (90.0%)

Total basepairs processed:   126,809,970 bp
  Read 1:    75,837,335 bp
  Read 2:    50,972,635 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):    104,332,265 bp (82.3%)
  Read 1:    63,680,774 bp
  Read 2:    40,651,491 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 241815 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	3.8	0	3
9	1	0.9	0	1
10

...

This is cutadapt 4.4 with Python 3.8.16
Command line parameters: --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -q 0,0 --quality-base 33 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-mcqlaf4n/S210421121705_64_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-mcqlaf4n/S210421121705_65_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2/lauro/data/1dff5a72-bbd8-47cf-8f76-5a4793729825/data/S210421121705_64_L001_R1_001.fastq.gz /tmp/qiime2/lauro/data/1dff5a72-bbd8-47cf-8f76-5a4793729825/data/S210421121705_65_L001_R2_001.fastq.gz
Processing paired-end reads on 6 cores ...


Finished in 1.353 s (3.928 µs/read; 15.28 M reads/minute).

=== Summary ===

Total read pairs processed:            344,337
  Read 1 with adapter:                 334,766 (97.2%)
  Read 2 with adapter:                 317,372 (92.2%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           35,670 (10.4%)
Pairs written (passing filters):       308,667 (89.6%)

Total basepairs processed:   175,611,870 bp
  Read 1:   105,022,785 bp
  Read 2:    70,589,085 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):    143,899,876 bp (81.9%)
  Read 1:    87,831,910 bp
  Read 2:    56,067,966 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 334766 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	1	5.3	0	1
9	4	1.3	0	4
10

Finished in 1.621 s (3.893 µs/read; 15.41 M reads/minute).

=== Summary ===

Total read pairs processed:            416,322
  Read 1 with adapter:                 405,067 (97.3%)
  Read 2 with adapter:                 385,489 (92.6%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           41,066 (9.9%)
Pairs written (passing filters):       375,256 (90.1%)

Total basepairs processed:   212,324,220 bp
  Read 1:   126,978,210 bp
  Read 2:    85,346,010 bp
Quality-trimmed:                       0 bp (0.0%)
  Read 1:             0 bp
  Read 2:             0 bp
Total written (filtered):    174,944,302 bp (82.4%)
  Read 1:   106,779,862 bp
  Read 2:    68,164,440 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 405067 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	4	6.4	0	4
9	2	1.6	0	2
10	

/home/lauro/anaconda3/envs/qiime2-2023.5/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)


/home/lauro/anaconda3/envs/qiime2-2023.5/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)
